Загрузите [файл со статьями статей из Википедии](https://yadi.sk/d/ObKNNcaFWEsK-w). Каждая строка в файле имеет следующий формат:

   >URL статьи `<tab>` название статьи `<tab>` текст 

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import split

# Создаем SparkSession
spark = SparkSession.builder \
    .appName("Data") \
    .getOrCreate()

# Считываем файл с данными
data = spark.read.text("C:/Users/MSI-PC/Desktop/my tipo dz po pitonu/task3-2024/wiki.txt")

# Разбиваем строки по табуляции и преобразуем их в три колонки
split_data = data.select(split(data.value, "\t").alias("columns"))

# Создаем DataFrame с тремя столбцами
split_data = split_data.selectExpr("columns[0] as URL", "columns[1] as title", "columns[2] as text")

split_data = split_data.filter(~split_data.text.rlike("http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+"))
split_data = split_data.filter(~split_data.text.rlike("ftp[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+"))

# Выводим полученный DataFrame
split_data.show(truncate=False)

# Завершаем SparkSession
spark.stop()

1. Напишите программу, которая находит самое длинное слово.

In [6]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import split, length, explode, max

# Создаем SparkSession
spark = SparkSession.builder \
    .appName("Data") \
    .getOrCreate()

# Считываем файл с данными
data = spark.read.text("C:/Users/MSI-PC/Desktop/my tipo dz po pitonu/task3-2024/wiki.txt")

# Разбиваем строки по табуляции и преобразуем их в три колонки
split_data = data.select(split(data.value, "\t").alias("columns"))

# Создаем DataFrame с тремя столбцами
split_data = split_data.selectExpr("columns[0] as url", "columns[1] as title", "columns[2] as text")

split_data = split_data.filter(~split_data.text.rlike("http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+"))
split_data = split_data.filter(~split_data.text.rlike("ftp[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+"))
# Выводим полученный DataFrame
# split_data.show(truncate=False)

# Сплитуем слова через пробел и ,
words = split_data.select(explode(split(split_data.text, "[\\s,]+")).alias("word"))

# Находим самое длинное слово
longest_word_row = words.filter(length("word") > 0).groupBy().agg(max(length("word")).alias("max_length")).collect()
longest_word_length = longest_word_row[0]["max_length"]
longest_word = words.filter(length("word") == longest_word_length).select("word").collect()[0]["word"]
print("Самое длинное слово :", longest_word)
print( len(longest_word))
# Завершаем SparkSession
spark.stop()


Самое длинное слово : Галогенарены-Нафталин-Антрацен-Фенантрен-Бензпирен-Коронен-Азулен-Бифенил-Ионол.
80


2. Напишите программу, которая находит среднюю длину слов. 

In [7]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import split, avg

# Создаем SparkSession
spark = SparkSession.builder \
    .appName("Data") \
    .getOrCreate()

# Считываем файл с данными
data = spark.read.text("C:/Users/MSI-PC/Desktop/my tipo dz po pitonu/task3-2024/wiki.txt")

# Разбиваем строки по табуляции и преобразуем их в три колонки
split_data = data.select(split(data.value, "\t").alias("columns"))


# Создаем DataFrame с тремя столбцами
split_data = split_data.selectExpr("columns[0] as url", "columns[1] as title", "columns[2] as text")

split_data = split_data.filter(~split_data.text.rlike("http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+"))
split_data = split_data.filter(~split_data.text.rlike("ftp[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+"))
# Выводим полученный DataFrame
#split_data.show(truncate=False)
words = split_data.select(explode(split(split_data.text, "[\\s,]+")).alias("word"))

average_word_length = words.filter(length("word") > 0).agg(avg(length("word")).alias("avg_length")).collect()[0]["avg_length"]
print("Средняя длина слова :", average_word_length)
# Завершaем SparkSession
spark.stop()

Средняя длина слова : 6.434972869181451


3. Напишите программу, которая находит самое частоупотребляемое слово, состоящее из латинских букв.

In [8]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import split
from pyspark.sql.functions import split, explode, desc, col

# Создаем SparkSession
spark = SparkSession.builder \
    .appName("Data") \
    .getOrCreate()

# Считываем файл с данными
data = spark.read.text("C:/Users/MSI-PC/Desktop/my tipo dz po pitonu/task3-2024/wiki.txt")

# Разбиваем строки по табуляции и преобразуем их в три колонки
split_data = data.select(split(data.value, "\t").alias("columns"))


# Создаем DataFrame с тремя столбцами
split_data = split_data.selectExpr("columns[0] as url", "columns[1] as title", "columns[2] as text")

split_data = split_data.filter(~split_data.text.rlike("http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+"))
# Выводим полученный DataFrame
#split_data.show(truncate=False)
words = split_data.select(explode(split(split_data.text, "[\\s,]+")).alias("word")).filter("word != ''").withColumn("word", col("word"))

# Ищем слова на латинице
latin_words = words.filter(words["word"].rlike("^[a-zA-Z]+$"))
most_common_latin_word = latin_words.groupBy("word").count().sort(desc("count")).first()["word"]

print("Самое частоупотребляемое слово на латинице :", most_common_latin_word)
# Завершaем SparkSession
spark.stop()


Самое частоупотребляемое слово на латинице : I


4. Все слова, которые более чем в половине случаев начинаются с большой буквы и встречаются больше 10 раз.

In [19]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import split, explode, col, length, desc

# Создаем SparkSession
spark = SparkSession.builder \
    .appName("Data") \
    .getOrCreate()

# Считываем файл с данными
data = spark.read.text("C:/Users/MSI-PC/Desktop/my tipo dz po pitonu/task3-2024/wiki.txt")

# Разбиваем строки по табуляции и преобразуем их в три колонки
split_data = data.select(split(data.value, "\t").alias("columns"))

# Создаем DataFrame с тремя столбцами
split_data = split_data.selectExpr("columns[0] as url", "columns[1] as article_title", "columns[2] as article_text")

# Фильтрация строк, содержащих URL
split_data = split_data.filter(~split_data.article_text.rlike("http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+"))
split_data = split_data.filter(~split_data.article_text.rlike("ftp[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+"))

# Разбиваем текст на слова и фильтруем их
words = split_data.select(explode(split(split_data.article_text, "[\\s,]+")).alias("word")).filter("word != ''")

# Фильтрация слов, начинающихся с большой буквы, не являющихся римскими цифрами и не состоящих из одной буквы
capitalized_words = words.filter((col("word").rlike("^[A-ZА-Я]")) & ~col("word").rlike("^(?i)([IVXLCDM]+)$") & (length(col("word")) > 1))

# Считаем количество упоминаний каждого слова
word_counts = words.groupBy("word").count().filter("count > 10")

# Фильтруем слова, которые встречаются более чем в половине случаев
total_words = words.count()
frequent_capitalized_words = word_counts.filter(word_counts["count"] > total_words / 2)

# Считаем количество упоминаний слов, начинающихся с большой буквы, исключая римские цифры и слова из одной буквы
frequent_capitalized_words = capitalized_words.groupBy("word").count().filter("count > 10").orderBy(col("count").desc())

# Выводим первые 5 строк
print("Слова, начинающиеся с большой буквы и встречающиеся больше 10 раз:")
frequent_capitalized_words.limit(5).show(truncate=False)

# Завершаем SparkSession
spark.stop()


Слова, начинающиеся с большой буквы и встречающиеся больше 10 раз:
+------+-----+
|word  |count|
+------+-----+
|На    |9362 |
|По    |8270 |
|России|6197 |
|После |6096 |
|Однако|5166 |
+------+-----+



In [15]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode, split, col

# Инициализация SparkSession
spark = SparkSession.builder.appName("AbbreviationAnalysis").getOrCreate()

# Загрузка данных
file_path = "C:/Users/MSI-PC/Desktop/my tipo dz po pitonu/task3-2024/wiki.txt"  
data = spark.read.csv(file_path, sep='\t', header=False)
data = data.withColumnRenamed('_c0', 'url').withColumnRenamed('_c1', 'title').withColumnRenamed('_c2', 'text')

# Разбиваем текст на слова
words = data.select(explode(split(col('text'), '\\s+')).alias('word'))

# Фильтруем слова для сокращений `пр.` и `др.`
abbreviations = words.filter(col('word').isin('пр.', 'др.')) \
    .groupBy('word') \
    .count() \
    .orderBy(col('count').desc())

# Отображаем результаты
abbreviations.show(truncate=False)

# Останавливаем SparkSession
spark.stop()


+----+-----+
|word|count|
+----+-----+
|др. |1276 |
|пр. |107  |
+----+-----+



In [20]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode, split, col

# Инициализация SparkSession
spark = SparkSession.builder.appName("AbbreviationAnalysis").getOrCreate()

# Загрузка данных
file_path = "C:/Users/MSI-PC/Desktop/my tipo dz po pitonu/task3-2024/wiki.txt"  
data = spark.read.csv(file_path, sep='\t', header=False)
data = data.withColumnRenamed('_c0', 'url').withColumnRenamed('_c1', 'title').withColumnRenamed('_c2', 'text')

# Разбиваем текст на слова
words = data.select(explode(split(col('text'), '\\s+')).alias('word'))

# Фильтруем слова для сокращений `т.п.` и `н.э.`
abbreviations = words.filter(col('word').isin('т.п.', 'н.э.')) \
    .groupBy('word') \
    .count() \
    .orderBy(col('count').desc())

# Выводим результаты
abbreviations.show(truncate=False)

# Останавливаем SparkSession
spark.stop()

+----+-----+
|word|count|
+----+-----+
|н.э.|44   |
|т.п.|17   |
+----+-----+

